In [1]:
import os
import sys
import time
import random

import numpy as np
import skimage.io as io
import mrcnn.model as modellib

from cv2 import cv2
from glob import glob
from mrcnn import utils
from samples.coco import coco
from threading import Thread
from skimage.util import img_as_float, img_as_ubyte

Using TensorFlow backend.


In [2]:
#check model, download if not exist
if not os.path.exists('mask_rcnn_coco.h5'):
    utils.download_trained_weights('mask_rcnn_coco.h5')

#create config
class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

#modelling
model = modellib.MaskRCNN(mode="inference", model_dir='logs/', config=config)

model.load_weights('mask_rcnn_coco.h5', by_name=True)

class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [3]:
rpn = modellib.build_rpn_model(config.RPN_ANCHOR_STRIDE, len(config.RPN_ANCHOR_RATIOS), config.TOP_DOWN_PYRAMID_SIZE)

In [5]:
rpn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_rpn_feature_map (InputLay (None, None, None, 2 0                                            
__________________________________________________________________________________________________
rpn_conv_shared (Conv2D)        (None, None, None, 5 1180160     input_rpn_feature_map[0][0]      
__________________________________________________________________________________________________
rpn_class_raw (Conv2D)          (None, None, None, 6 3078        rpn_conv_shared[0][0]            
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, None, 2)      0           rpn_class_raw[0][0]              
__________________________________________________________________________________________________
rpn_bbox_p

In [6]:
from keras.utils.vis_utils import plot_model

plot_model(rpn, to_file='rpn_plot.png', show_shapes=True, show_layer_names=True)

In [5]:
files = glob('videos/*')
master = None

def worker(frame, func):
    global master
    now = time.time()
    master = func([frame], verbose=1)
    end = time.time() - now
    master[0]['time'] = end

for video in files:
    name = video.split('\\')[1]
    name = name.split('.')[0]
    capturer = cv2.VideoCapture(video)

    counter = 0
    while True:
        counter += 1
        ret, frame = capturer.read()
        time.sleep(1/30)
        if not ret:
            break

        # now = time.time()
        # master = model.detect([frame])
        # end = time.time() - now
        # master[0]['time'] = end

        if counter == 1:
            now = time.time()
            master = model.detect([frame])
            end = time.time() - now
            master[0]['time'] = end

        if counter % 20 == 0:
            t = Thread(target=worker, args=(frame, model.detect))
            t.start()

        if master != None:
            result = master[0]
            for i in range(len(result['rois'])):
                y1, x1, y2, x2 = result['rois'][i]
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 1)
                cv2.putText(frame, class_names[result['class_ids'][i]], (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                cv2.putText(frame, str(result['time']), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        cv2.imshow('video', frame)
        cv2.waitKey(1)
        cv2.imwrite('output/{}_{}.png'.format(counter, name), frame)

    capturer.release()
    cv2.destroyAllWindows()

Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-b156f1028093>", line 7, in worker
    master = func([frame], verbose=1)
  File "D:\kuliah\TA\ta_mask_rcnn\mrcnn\model.py", line 2524, in detect
    self.keras_model.predict([molded_images, image_metas, anchors], verbose=0)
  File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1462, in predict
    callbacks=callbacks)
  File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training_arrays.py", line 324, in predict_loop
    batch_outs = f(ins_batch)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\backend.py", line 3473, in __call__
    self._make_callable(feed_arrays, feed_symbols, symbol_vals, session)
  File "C:\ProgramD

Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-b156f1028093>", line 7, in worker
    master = func([frame], verbose=1)
  File "D:\kuliah\TA\ta_mask_rcnn\mrcnn\model.py", line 2524, in detect
    self.keras_model.predict([molded_images, image_metas, anchors], verbose=0)
  File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1462, in predict
    callbacks=callbacks)
  File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training_arrays.py", line 324, in predict_loop
    batch_outs = f(ins_batch)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\backend.py", line 3473, in __call__
    self._make_callable(feed_arrays, feed_symbols, symbol_vals, session)
  File "C:\ProgramD

Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


...

Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


Exception in thread Thread-14:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-b156f1028093>", line 7, in worker
    master = func([frame], verbose=1)
  File "D:\kuliah\TA\ta_mask_rcnn\mrcnn\model.py", line 2524, in detect
    self.keras_model.predict([molded_images, image_metas, anchors], verbose=0)
  File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1462, in predict
    callbacks=callbacks)
  File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training_arrays.py", line 324, in predict_loop
    batch_outs = f(ins_batch)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\backend.py", line 3473, in __call__
    self._make_callable(feed_arrays, feed_symbols, symbol_vals, session)
  File "C:\ProgramD

Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


Exception in thread Thread-15:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-b156f1028093>", line 7, in worker
    master = func([frame], verbose=1)
  File "D:\kuliah\TA\ta_mask_rcnn\mrcnn\model.py", line 2524, in detect
    self.keras_model.predict([molded_images, image_metas, anchors], verbose=0)
  File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1462, in predict
    callbacks=callbacks)
  File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training_arrays.py", line 324, in predict_loop
    batch_outs = f(ins_batch)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\backend.py", line 3473, in __call__
    self._make_callable(feed_arrays, feed_symbols, symbol_vals, session)
  File "C:\ProgramD

KeyboardInterrupt: 